In [2]:
!nvidia-smi

Wed Oct  7 12:29:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.30                 Driver Version: 390.30                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    60W / 149W |      0MiB / 11441MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
#htop 

In [1]:
#paths required
datafolder_p = r'D:\18-DS\github\SDSHL\data\processed' #Change this when working on Nimblebox
datafolder_e = r'D:\18-DS\github\SDSHL\data\external'
datafolder_i = r'D:\18-DS\github\SDSHL\data\internal'
modelfolder  = r'D:\18-DS\data\models\BERT'
resultsfolder = r'D:\18-DS\github\SDSHL\data\results'

# vectorfolder_bert = modelfolder + r'\BERT'
# vectorfolder_ft   = modelfolder + r'\fasttext_wiki.hi'
# vectorfolder_standford = modelfolder + r'\POS-Tagger-Hindi'

filepath_train =datafolder_p+ r'\2-train.csv' #Change this when working on Nimblebox
filepath_test  =datafolder_p+ r'\2-test.csv'  #Change this when working on Nimblebox
filepath_fulldata = datafolder_p+ r'\2-Hinglish_Sarcasm_Clean.csv'

In [1]:
import pandas as pd
import numpy as np
import sys
import random as rn

import sklearn.metrics as metrics
from sklearn.naive_bayes import GaussianNB
#import sklearn.svm as svm
from sklearn.linear_model import LogisticRegression             
from sklearn.pipeline import Pipeline

from keras.preprocessing.sequence import pad_sequences

import torch
from torch import nn
from pytorch_pretrained_bert import BertModel
from pytorch_pretrained_bert import BertTokenizer

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
#from torchnlp.datasets import imdb_dataset

from IPython.display import clear_output

ImportError: cannot import name 'get_config' from 'tensorflow.python.eager.context' (d:\Users\admin\anaconda3\envs\myenv\lib\site-packages\tensorflow_core\python\eager\context.py)

In [3]:
df_train = pd.read_csv(filepath_train, sep="\t")
df_test  = pd.read_csv(filepath_test, sep="\t")
df_full  = pd.read_csv(filepath_fulldata, sep="\t", index_col="ID")

X_train =  df_train['sentence'] 
y_train = df_train['label']

X_test =  df_test['sentence'] 
y_test = df_test['label']



In [4]:
prediction={}

## Prepare Tensor Dataset of Train and Test Dataset

In [5]:
X_train = list(X_train)
y_train = list(y_train)

X_test = list(X_test)
y_test = list(y_test)

len(X_train), len(y_train), len(X_test), len(y_test)

(1800, 1800, 200, 200)

In [ ]:
#!pip install transformers

## Common Functions

In [6]:
from typing import Callable, List, Optional, Tuple

import pandas as pd
from sklearn.base import TransformerMixin, BaseEstimator
import torch


class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(
            self,
            bert_tokenizer,
            bert_model,
            max_length: int = 60,
            embedding_func: Optional[Callable[[torch.tensor], torch.tensor]] = None,
    ):
        self.tokenizer = bert_tokenizer
        self.model = bert_model
        self.model.eval()
        self.max_length = max_length
        self.embedding_func = embedding_func

        if self.embedding_func is None:
            self.embedding_func = lambda x: x[0][:, 0, :].squeeze()

    def _tokenize(self, text: str) -> Tuple[torch.tensor, torch.tensor]:
        # Tokenize the text with the provided tokenizer
        tokenized_text = self.tokenizer.encode_plus(text,
                                                    add_special_tokens=True,
                                                    max_length=self.max_length,
                                                    truncation=True
                                                    )["input_ids"]

        # Create an attention mask telling BERT to use all words
        attention_mask = [1] * len(tokenized_text)

        # bert takes in a batch so we need to unsqueeze the rows
        return (
            torch.tensor(tokenized_text).unsqueeze(0),
            torch.tensor(attention_mask).unsqueeze(0),
        )

    def _tokenize_and_predict(self, text: str) -> torch.tensor:
        tokenized, attention_mask = self._tokenize(text)

        embeddings = self.model(tokenized, attention_mask)
        return self.embedding_func(embeddings)

    def transform(self, text: List[str]):
        if isinstance(text, pd.Series):
            text = text.tolist()

        with torch.no_grad():
            return torch.stack([self._tokenize_and_predict(string) for string in text])

    def fit(self, X, y=None):
        """No fitting necessary so we just return ourselves"""
        return self

In [7]:
def pred_for_test(model):
    y_pred=[]
    for i in range(len(X_test)):
        p=model.predict(X_test.iloc[i])
        x1 = int(p[0][-1][-1])
        x2 = p[1][0]
        if x1==0: x2= 1-x2
        y_pred.append( x2 )
    return y_pred    


def print_metrics(y_pred):
    threshold=0.5
    y_pred1=[]
    for i in y_pred :
        if i>threshold:
            y_pred1.append(1)
        else:
            y_pred1.append(0)

    acc = np.round( metrics.accuracy_score(y_test,y_pred1), 2)
    recall = np.round( metrics.recall_score(y_test,y_pred1), 2)
    precision = np.round( metrics.precision_score(y_test,y_pred1), 2)
    f1 = np.round( metrics.f1_score(y_test,y_pred1), 2)
    roc = np.round( metrics.roc_auc_score(y_test,y_pred1), 2)

    print("Accuracy : ", acc )
    print("Recall   : ", recall )
    print("Precision: ", precision )
    print("F1       : ", f1 )
    print("ROC      : ", roc )

# <font color=red> BERT Multilanguage - Task Transfer - Pytorch</font>
<font color=red> pytoch works only on Nimblebox. Because it need GPU and my machine does not have that. So DON'T RUN it Here</font>

In [ ]:
#Inspiration from
#https://towardsdatascience.com/bert-to-the-rescue-17671379687f
#https://github.com/shudima/notebooks/blob/master/BERT_to_the_rescue.ipynb

In [8]:
def get_bert_tokens():
    #This will download Pretrained Multilingual Model and create a Model in Memory
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

    train_tokens = list(map(lambda t: ['[CLS]'] + bert_tokenizer.tokenize(t)[:255] + ['[SEP]'], X_train))
    test_tokens = list(map(lambda t: ['[CLS]'] + bert_tokenizer.tokenize(t)[:255] + ['[SEP]'], X_test))

    print ("Tokens:", len(train_tokens), len(test_tokens) )

    train_tokens_ids = pad_sequences(list(map(bert_tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=256, truncating="post", padding="post", dtype="int")
    test_tokens_ids = pad_sequences(list(map(bert_tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=256, truncating="post", padding="post", dtype="int")

    print ("Token Ids:",train_tokens_ids.shape, test_tokens_ids.shape)

    train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
    test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

    print ("Masks: ", len(train_masks), len(test_masks))
    return train_tokens, test_tokens, train_tokens_ids, test_tokens_ids, train_masks, test_masks


In [33]:
BATCH_SIZE = 2
EPOCHS = 10
def get_tensor_dataset(train_tokens_ids,test_tokens_ids,y_train,y_test, train_masks, test_masks):
    train_tokens_tensor = torch.tensor(train_tokens_ids)
    train_y_tensor = torch.tensor(y_train).float()

    test_tokens_tensor = torch.tensor(test_tokens_ids)
    test_y_tensor = torch.tensor(y_test).float()

    train_masks_tensor = torch.tensor(train_masks)
    test_masks_tensor = torch.tensor(test_masks)

    print ( str(torch.cuda.memory_allocated(torch.cuda.current_device())/1000000 ) + 'M')

    train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
    #train_sampler = RandomSampler(train_dataset)
    #train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)
    train_dataloader = DataLoader(train_dataset,  batch_size=BATCH_SIZE)

    test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
    #test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    
    return train_dataloader, test_dataloader


In [34]:
train_tokens, test_tokens, train_tokens_ids, test_tokens_ids, train_masks, test_masks = get_bert_tokens()

Tokens: 1800 200
Token Ids: (1800, 256) (200, 256)
Masks:  1800 200


In [35]:
train_dataloader, test_dataloader  = \
get_tensor_dataset(train_tokens_ids,test_tokens_ids,y_train,y_test, train_masks, test_masks)

0.014848M


### Develop BERT Multilanguage Model

In [9]:
#This is required for multilingual bert

class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained( 'bert-base-multilingual-uncased')  

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        #_, pooled_output = self.bert(tokens, attention_mask=masks)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [39]:
bert_clf = BertBinaryClassifier()
bert_clf = bert_clf.cuda()

In [40]:
param_optimizer = list(bert_clf.sigmoid.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [31]:
def train_bert():
    for epoch_num in range(EPOCHS):
        bert_clf.train()
        train_loss = 0
        for step_num, batch_data in enumerate(train_dataloader):
            token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
            #print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
            logits = bert_clf(token_ids, masks)

            loss_func = nn.BCELoss()

            batch_loss = loss_func(logits, labels)
            train_loss += batch_loss.item()


            bert_clf.zero_grad()
            batch_loss.backward()


            clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
            optimizer.step()

            clear_output(wait=True)
            print('Epoch: ', epoch_num + 1)
            print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(token_ids) / BATCH_SIZE, train_loss / (step_num + 1)))
            
        return bert_clf

In [47]:
bert_clf = train_bert()
bert_clf = bert_clf.cuda()

Epoch:  1

899/1.0 loss: 0.6341746076279217 


### BERT Multilanguage - Evaluation

In [42]:
def evaluate_model():
    bert_clf.eval()
    bert_predicted = []
    all_logits = []
    with torch.no_grad():
        for step_num, batch_data in enumerate(test_dataloader):

            token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

            logits = bert_clf(token_ids, masks)
            loss_func = nn.BCELoss()
            loss = loss_func(logits, labels)
            numpy_logits = logits.cpu().detach().numpy()

            #bert_predicted += list(numpy_logits[:, 0] > 0.5)
            bert_predicted += list(numpy_logits[:, 0])
            
    return bert_predicted

In [48]:
bert_predicted_multi = evaluate_model()

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


In [ ]:
from sklearn.metrics import classification_report
prediction['mBERT_PytorchTT'] = bert_predicted_multi

print(classification_report(y_test, np.array(bert_predicted_multi)>.5))

# <font color=red> mBERT / Multilanguage - Task Transfer - Transformer</font>

In [10]:
from transformers import BertTokenizer as BertTokenizer2, BertModel as BertModel2
bert_model_multi     = BertModel2.from_pretrained(modelfolder + r'\bert-base-multilingual-uncased')
bert_tokenizer_multi = BertTokenizer2.from_pretrained(modelfolder + r'\bert-base-multilingual-uncased')

In [11]:
bert_transformer_multi = BertTransformer(bert_tokenizer_multi, bert_model_multi, max_length=200)
#classifier = svm.LinearSVC(C=1.0, class_weight="balanced")
#classifier = LogisticRegression()
classifier = GaussianNB()

model_pipeline_multi = Pipeline(
    [
        ("vectorizer", bert_transformer_multi),
        ("classifier", classifier),
    ]
)
model_pipeline_multi.fit(X_train, y_train)

D:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


Pipeline(steps=[('vectorizer',
                 BertTransformer(bert_model=None, bert_tokenizer=None,
                                 embedding_func=<function BertTransformer.__init__.<locals>.<lambda> at 0x000001553D5AED30>,
                                 max_length=200)),
                ('classifier', GaussianNB())])

### mBERT  /Multilanguage - Transformer - Evaluation

In [12]:
y_pred_multi = model_pipeline_multi.predict_proba(X_test)

In [35]:
prediction['mBERT_TransformerTT'] = y_pred_multi[:,1]
print_metrics(y_pred_multi[:,1])

Accuracy :  0.58
Recall   :  0.65
Precision:  0.57
F1       :  0.61
ROC      :  0.58


# <font color=red> mBERT (Multilanguage) - Embedding Transfer</font>

In [14]:
#This steps takes approx 5 min.

import torch

mbert_embedding=[]
for i in range(len(df_full)):
    txt = df_full.iloc[i,:]['sentence']
    token_dict = bert_tokenizer_multi.encode_plus(txt, add_special_tokens=False)
    embed = bert_model_multi((torch.tensor(token_dict["input_ids"]).unsqueeze(0)))
    embed = embed[1][0].detach().numpy()
    #print (i, sep='\t')
    mbert_embedding.append(embed)

In [15]:
df_mbert = pd.DataFrame(mbert_embedding, index=df_full.index)
df_mbert['label'] = df_full['label']
df_mbert.head(5)

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,label
ID,,,,,,,,,,,,,,,,,,,,,
1,-0.094827,-0.215950,0.002612,0.234672,0.196442,0.032061,0.441763,-0.226298,-0.364357,0.092771,...,-0.241572,-0.193821,-0.136545,0.077711,0.382187,-0.238862,0.214421,-0.165181,-0.177810,1
5,-0.038516,-0.242589,-0.016740,0.177583,0.259062,-0.056287,0.393562,-0.124342,-0.305216,-0.001444,...,-0.242798,-0.161236,-0.138323,0.054489,0.389266,-0.065651,0.266223,-0.181780,-0.205100,1
12,-0.049239,-0.187880,-0.150780,0.173796,0.074700,-0.029632,0.253110,-0.011287,-0.269391,0.044454,...,-0.098685,-0.133191,-0.070416,-0.030003,0.250877,-0.089087,0.122501,-0.123521,-0.141871,1
13,-0.091381,-0.213047,-0.000264,0.234689,0.212581,0.039017,0.450925,-0.237699,-0.366408,0.108367,...,-0.252267,-0.209274,-0.139032,0.090818,0.392109,-0.244524,0.209654,-0.160693,-0.187955,1
26,-0.103511,-0.217064,0.008468,0.232815,0.178900,0.027574,0.430879,-0.205967,-0.356071,0.073888,...,-0.231867,-0.176149,-0.130337,0.062329,0.371234,-0.230683,0.218179,-0.172345,-0.170212,1


#### Save BERT Multilangue Embedding

In [16]:
df_mbert.to_csv(datafolder_p+r'\embedding_bert768_mbert_pretrained.csv')

# <font color=red> IndicBERT -Task Transfer - Transformer</font>

In [17]:
from transformers import AutoTokenizer, AutoModel
bert_tokenizer_indic = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
bert_model_indic = AutoModel.from_pretrained("ai4bharat/indic-bert")

In [18]:

bert_transformer_indic = BertTransformer(bert_tokenizer_indic, bert_model_indic, max_length=200)
#classifier = svm.LinearSVC(C=1.0, class_weight="balanced")
#classifier = LogisticRegression()
classifier = GaussianNB()

model_pipeline_indic = Pipeline(
    [
        ("vectorizer", bert_transformer_indic),
        ("classifier", classifier),
    ]
)
model_pipeline_indic.fit(X_train, y_train)

D:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


Pipeline(steps=[('vectorizer',
                 BertTransformer(bert_model=None, bert_tokenizer=None,
                                 embedding_func=<function BertTransformer.__init__.<locals>.<lambda> at 0x0000015534EBDA60>,
                                 max_length=200)),
                ('classifier', GaussianNB())])

### IndicBERT  - Evaluation

In [19]:
y_pred_indic = model_pipeline_indic.predict_proba(X_test)

In [36]:
prediction['IndicBERT_TT'] = y_pred_indic[:,1]
print_metrics(y_pred_indic[:,1])

Accuracy :  0.58
Recall   :  0.63
Precision:  0.57
F1       :  0.6
ROC      :  0.58


# <font color=red> IndicBERT - Embedding Transfer</font>

In [24]:
#This steps takes approx 5 min.

indicbert_embedding=[]
for i in range(len(df_full)):
    txt = df_full.iloc[i,:]['sentence']
    token_dict = bert_tokenizer_indic.encode_plus(txt, add_special_tokens=False, max_length=200, truncation=True)
    embed = bert_model_indic((torch.tensor(token_dict["input_ids"]).unsqueeze(0)))
    embed = embed[1][0].detach().numpy()
    #print (i, sep='\t')
    indicbert_embedding.append(embed)

In [25]:
df_indicbert = pd.DataFrame(indicbert_embedding, index=df_full.index)
df_indicbert['label'] = df_full['label']
df_indicbert.head(5)

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,label
ID,,,,,,,,,,,,,,,,,,,,,
1,-0.053525,0.054395,0.017980,-0.003695,0.040563,0.080847,0.035940,-0.005091,-0.018056,0.050841,...,-0.031083,0.057919,0.128374,-0.057468,-0.054164,-0.063550,-0.030812,0.034091,0.015657,1
5,-0.054423,0.050057,0.026642,0.001021,0.040388,0.084593,0.040480,-0.006151,-0.018039,0.049458,...,-0.027824,0.066910,0.133481,-0.054165,-0.059787,-0.067824,-0.032006,0.032855,0.019553,1
12,-0.052677,0.054255,0.020152,0.000721,0.038358,0.079170,0.039753,-0.004033,-0.014087,0.051323,...,-0.030897,0.061226,0.132689,-0.061606,-0.064817,-0.065052,-0.039030,0.036289,0.016919,1
13,-0.056458,0.047421,0.008888,-0.004903,0.043382,0.084137,0.030783,-0.006145,-0.014157,0.050950,...,-0.025582,0.060028,0.126639,-0.054046,-0.056553,-0.067852,-0.035591,0.026892,0.020892,1
26,-0.055969,0.045073,0.028485,-0.003376,0.040905,0.086122,0.036461,-0.003613,-0.019812,0.051705,...,-0.036923,0.058726,0.134128,-0.059550,-0.056367,-0.062027,-0.031240,0.032978,0.009224,1


#### Save BERT indicnlp Embedding

In [26]:
df_indicbert.to_csv(datafolder_p+r'\embedding_bert768_indicbert_pretrained.csv')

### <font color=red> Save Task Transfer Prediction Results of mBERT and indicBERT</font>

In [37]:
df_prediction= pd.DataFrame(prediction, index=df_test.ID)
df_prediction.to_csv(resultsfolder+ r'\model_predictions_BERT.csv')
df_prediction

,mBERT_TransformerTT,IndicBERT_TransformerTT
ID,,
5212,3.900909e-01,4.119921e-40
8028,1.000000e+00,1.000000e+00
2364,1.000000e+00,1.000000e+00
5805,2.755473e-04,2.135327e-19
5236,1.000000e+00,5.874681e-03
...,...,...
7171,3.214948e-50,5.854190e-09
8819,1.000000e+00,1.000000e+00
2686,1.259107e-38,2.715738e-32
